# Pipeline

[Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) é uma classe do sklearn que permite aplicar uma sequência de transformações em um estimador final. <br>
Para isso, os passos intermediários devem ter implementados métodos de `fit` e `transform` enquanto o estimador final só precisa ter o `fit` implementado. <br>
O propósito do `pipeline` é:
- reunir várias etapas para serem validadas de forma cruzada (cross-validation) ao definir parâmetros diferentes
- ajudar a criar códigos que possuam um padrão que possa ser facilmente entendido e compartilhando entre times de cientista e engenheiro de dados.

<img src="images/pipeline.png" text="https://nbviewer.org/github/rasbt/python-machine-learning-book/blob/master/code/ch06/ch06.ipynb#Combining-transformers-and-estimators-in-a-pipeline">



## Machine learning pipeline

Como é a sequência de processos necessários para implementar um modelo de Machine Learning?
<br><br>
<div>
    <img src="https://s3-sa-east-1.amazonaws.com/lcpi/aa5c334b-5b94-49da-b8b5-43966a5b87d0.png" width=700>
</div>
<br><br>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Continuamos pegando o dataset iris, que é simples e já vimos ele antes.
# Mas agora, eu adicionei alguns espaços nulos aleatoriamente.
df = pd.read_csv('../data/iris_with_nulls.csv')

In [4]:
# Vamos ver as dimensões dele
df.shape

(150, 5)

In [5]:
# Observe que cada feature tem pelo menos 1 nulo (afinal, non-null delas é menor que 150)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  146 non-null    float64
 1   sepal_width   146 non-null    float64
 2   petal_length  149 non-null    float64
 3   petal_width   144 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [6]:
# Confirmando quantos nulos temos em cada coluna
df.isnull().sum()

sepal_length    4
sepal_width     4
petal_length    1
petal_width     6
species         0
dtype: int64

Eu vou querer, para rodar meu algoritmo de machine learning, transformar minha "target" (que é "species"), em um número ordinal, (0, 1 ou 2). Eu posso rodar isso no treino e na validação, sem muito problema.
<br><br><br>
Já para as minhas features, eu quero seguir os seguintes passos de pré-processamento,
1. "padronizar" as minhas features (ou "normalizar", deixar elas com média 0 e desvio padrão 1),
2. adicionar a mediana em qualquer valor nulo,
3. aí sim, rodar meu modelo.

Neste caso, note que eu preciso "treinar" os passos (1), (2) e (3) todos na base de treino, e depois só aplicar eles na base de validação.

In [7]:
# vamos primeiro separar em features e target
X = df.drop(columns=['species'])
y = df.species

In [8]:
# Agora, vamos transformar as labels em números, 0, 1 ou 2
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_enc = label_encoder.fit_transform(y)

In [9]:
print(y[:10])
print('')
print(y_enc[:10])

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
5    Iris-setosa
6    Iris-setosa
7    Iris-setosa
8    Iris-setosa
9    Iris-setosa
Name: species, dtype: object

[0 0 0 0 0 0 0 0 0 0]


In [10]:
# Para os passos de processamento das features, faremos todos juntos, com um Pipeline.
from sklearn.pipeline import Pipeline

In [11]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# O SimpleImputer vai me ajudar a colocar a mediana.
# O StandardScaler vai padronizar as features.
# Meu modelo vai ser o KNN.

Como utilizar: <br>
O Pipeline é construído com uma lista de pares (key, value) nos quais a key é uma string que contém um nome para o step escolhido e o valor é o objeto da classe:

In [12]:
pipe_knn = Pipeline([('scaler', StandardScaler()),
                     ('imputer', SimpleImputer(strategy='median')),
                     ('model', KNeighborsClassifier(n_neighbors=7))])

Os passos do pipeline podem ser acessados pelos índices ou passando a key:

In [13]:
pipe_knn[0]

StandardScaler()

In [14]:
pipe_knn['imputer']

SimpleImputer(strategy='median')

In [15]:
# Podemos visualizar nosso pipe
from sklearn import set_config
set_config(display="diagram")
pipe_knn  # click on the diagram below to see the details of each step

Pipeline(steps=[('scaler', StandardScaler()),
                ('imputer', SimpleImputer(strategy='median')),
                ('model', KNeighborsClassifier(n_neighbors=7))])

In [16]:
# Agora vamos separar em treino e validação - Estratégia "Holdout set"
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y_enc, test_size=0.3, stratify=y_enc)

In [17]:
# Vamos rodar nosso pipeline no treino
pipe_knn.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('imputer', SimpleImputer(strategy='median')),
                ('model', KNeighborsClassifier(n_neighbors=7))])

In [18]:
# O que ele fez no fim, afinal??? Bem, ele aplicou os passos que pedimos.

# Primeiro, o Standard scaler. Ele viu como fazer média 0 e variância 1. Agora ele pode só transformar.
X_transformado = pipe_knn.named_steps['scaler'].transform(X_train)

# Nota que ainda teremos nulos.
print(np.isnan(X_transformado).sum())

10


In [19]:
# Depois, ele achou qual a mediana dos dados. O SimpleImputer pode então preencher os nulos.
X_sem_nulos = pipe_knn.named_steps['imputer'].transform(X_transformado)

# Nota que não teremos nulos.
print(np.isnan(X_sem_nulos).sum())

0


In [20]:
# Por fim, ele treinou o KNN no resultado. Assim, o KNN pode só usar o predict.
y_pred_train = pipe_knn.named_steps['model'].predict(X_sem_nulos)

print(y_pred_train[:10])

[0 0 1 1 2 2 0 0 1 2]


In [21]:
# De uma vez só:
y_pred_train = pipe_knn.predict(X_train)

print(y_pred_train[:10])

[0 0 1 1 2 2 0 0 1 2]


In [22]:
# Agora avaliamos o modelo no nosso conjunto de validação.
from sklearn.metrics import classification_report

y_pred_val = pipe_knn.predict(X_val)

print(classification_report(y_val, y_pred_val))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00        15
           2       1.00      1.00      1.00        15

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



Por que isso é poderoso?

Imagina se quiséssemos fazer validação cruzada. Nesse caso, para cada fold que escolhermos como validação, teriamos que refazer cada passo na "base de treino da vez",
1. achar a média e a variância daquela base de treino, para padronizar as features,
2. achar a mediana daquela base de treino, e preencher os nulos,
3. aí sim, treinamos o modelo na base de treino, e medimos a qualidade no fold de validação da vez.

Mas usando Pipeline, a gente não precisa fazer tudo passo a passo. O Pipeline se encarrega de fazer tudo de uma vez para nós. É como se o nosso "modelo" agora fosse o pipeline completo. 

In [23]:
pipe_knn_cv = Pipeline([('scaler_cv', StandardScaler()),
                        ('imputer_cv', SimpleImputer(strategy='median')),
                        ('model_cv', KNeighborsClassifier(n_neighbors=7))])

In [24]:
from sklearn.model_selection import cross_validate

Fazendo um exemplo mais simples: Queremos apenas a acurácia (que é global, não é uma pra cada classe)

In [25]:
cross_validate(pipe_knn_cv, X, y_enc, scoring='accuracy', cv=5)

{'fit_time': array([0.00355434, 0.00307631, 0.00356078, 0.00252557, 0.00286222]),
 'score_time': array([0.00232053, 0.00266004, 0.00172997, 0.00187612, 0.00191689]),
 'test_score': array([0.96666667, 0.96666667, 0.93333333, 0.9       , 1.        ])}

Fazendo um exemplo mais complexo: Eu quero a precision e o recall de cada classe.

In [26]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, make_scorer

In [27]:
scoring = {'Precision label 0':make_scorer(precision_score, labels=[0], average='micro'),
           'Precision label 1':make_scorer(precision_score, labels=[1], average='micro'),
           'Precision label 2':make_scorer(precision_score, labels=[2], average='micro'),
           'Recall label 0':make_scorer(recall_score, labels=[0], average='micro'),
           'Recall label 1':make_scorer(recall_score, labels=[1], average='micro'),
           'Recall label 2':make_scorer(recall_score, labels=[2], average='micro')}

In [28]:
cross_validate(pipe_knn_cv, X, y_enc, scoring=scoring, cv=5)

{'fit_time': array([0.00326705, 0.00300932, 0.00296378, 0.00245476, 0.00304914]),
 'score_time': array([0.00394011, 0.00484538, 0.00334835, 0.00374985, 0.00359035]),
 'test_Precision label 0': array([1., 1., 1., 1., 1.]),
 'test_Precision label 1': array([0.90909091, 0.90909091, 1.        , 0.81818182, 1.        ]),
 'test_Precision label 2': array([1.        , 1.        , 0.83333333, 0.88888889, 1.        ]),
 'test_Recall label 0': array([1., 1., 1., 1., 1.]),
 'test_Recall label 1': array([1. , 1. , 0.8, 0.9, 1. ]),
 'test_Recall label 2': array([0.9, 0.9, 1. , 0.8, 1. ])}

## Bibliografia e Aprofundamento
- [Python Machine Learning Book](https://github.com/rasbt/python-machine-learning-book-3rd-edition)
- [Documentação](https://scikit-learn.org/stable/modules/compose.html)
- [ColumnTransformer](https://scikit-learn.org/stable/modules/compose.html#columntransformer-for-heterogeneous-data)
- [FeatureUnion](https://scikit-learn.org/stable/modules/compose.html#featureunion-composite-feature-spaces)